In [28]:
from pulp import *
import pandas as pd
import os 
path = os.getcwd()

In [29]:
param = pd.read_excel('{0}/Production_scheduling.xlsx'.format(path))

ipykernel_launcher:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.


In [30]:
param = param.rename(columns={'Unnamed: 0': 'period'})

In [31]:
param['t'] = range(1,13)
param['Capacity'] = 5000

In [32]:
param = param.set_index('t')

In [33]:
inventory = LpVariable.dicts('inv', [0,1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Integer')
inventory[0] = 200

In [34]:
production = LpVariable.dicts('Prod', [1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Integer')
binary = LpVariable.dicts('binary', [1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Binary')

In [35]:
time = [1,2,3,4,5,6,7,8,9,10,11,12]

In [36]:
model = LpProblem('Production', LpMinimize)

In [37]:
model += lpSum([inventory[t] * param.loc[t, 'storage cost'] + production[t] * param.loc[t,'var'] + binary[t] * param.loc[t, 'fixed cost'] for t in time])

In [38]:
for t in time:
    model += production[t] - inventory[t] + inventory[t-1]>= param.loc[t, 'demand']
    model += production[t] <= binary[t]* param.loc[t, 'Capacity']


In [39]:
model.solve()

1

In [40]:
for i in production:
    print(production[i], production[i].varValue)

Prod_1 2800.0
Prod_2 4000.0
Prod_3 2600.0
Prod_4 5000.0
Prod_5 5000.0
Prod_6 0.0
Prod_7 1250.0
Prod_8 5000.0
Prod_9 5000.0
Prod_10 1700.0
Prod_11 0.0
Prod_12 1200.0
